### 1 Import the Data

We will need the following libraries

In [1]:
from io import BytesIO
from zipfile import ZipFile
import pandas as pd
import requests
import numpy as np

url = "https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Downloads/Inpatient_Data_2011_CSV.zip"
content = requests.get(url)
zf = ZipFile(BytesIO(content.content))

for item in zf.namelist():
    print("File in zip: "+  item)

File in zip: Medicare_Hospital_Inpatient_PUF_Methodology_2014-05-30.pdf
File in zip: Medicare_Provider_Charge_Inpatient_DRG100_FY2011.csv


### 2 Clean Data

In [2]:
# the first line of the file contains a string - that line shall be ignored
df = pd.read_csv(zf.open('Medicare_Provider_Charge_Inpatient_DRG100_FY2011.csv'))
df.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region (HRR) Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,32963.07692,5777.241758,4763.736264
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,15131.85714,5787.571429,4976.714286
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,37560.37500,5434.958333,4453.791667
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,13998.28000,5417.560000,4129.160000
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,31633.27778,5658.333333,4851.444444


In [3]:
# rename columns to simplify
df = df.rename(columns={'DRG Definition': 'DRG', 'Provider Id': 'ProviderID','Provider Name': 'ProviderName','Provider Street Address': 'Address','Provider City': 'City','Provider State': 'State','Provider Zip Code': 'ZipCode','Total Discharges': 'Total'})
df.head()

,DRG,ProviderID,ProviderName,Address,City,State,ZipCode,Hospital Referral Region (HRR) Description,Total,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,32963.07692,5777.241758,4763.736264
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,15131.85714,5787.571429,4976.714286
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,37560.37500,5434.958333,4453.791667
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,13998.28000,5417.560000,4129.160000
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,31633.27778,5658.333333,4851.444444


In [4]:
# drop columns that will not be used in the analysis
DRG_df = df.drop(df.columns[[7,9,10,11]], axis=1)
DRG_df.head()

,DRG,ProviderID,ProviderName,Address,City,State,ZipCode,Total
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,91
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,14
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,24
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,25
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,18


In [5]:
# filter data by DRG and State
rslt_df = DRG_df[(DRG_df['State']=='MN') & (DRG_df['DRG']=='897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REHABILITATION THERAPY W/O MCC')]
rslt_df.shape

(21, 8)

In [6]:
# sort in descending order, and resent the index
data=rslt_df.sort_values(['Total'], ascending=[False])

data.reset_index(drop=True, inplace=True)
data

,DRG,ProviderID,ProviderName,Address,City,State,ZipCode,Total
0,897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REH...,240080,"UNIVERSITY OF MINNESOTA MEDICAL CENTER, FAIRVIEW",2450 RIVERSIDE AVENUE,MINNEAPOLIS,MN,55454,138
1,897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REH...,240132,UNITY HOSPITAL,550 OSBORNE ROAD,FRIDLEY,MN,55432,65
2,897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REH...,240115,MERCY HOSPITAL,4050 COON RAPIDS BLVD,COON RAPIDS,MN,55433,53
3,897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REH...,240038,UNITED HOSPITAL,333 NORTH SMITH,SAINT PAUL,MN,55102,48
4,897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REH...,240063,ST JOSEPH'S HOSPITAL,45 WEST 10TH STREET,SAINT PAUL,MN,55102,45
5,897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REH...,240106,REGIONS HOSPITAL,640 JACKSON STREET,SAINT PAUL,MN,55101,45
6,897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REH...,240010,MAYO CLINIC - SAINT MARYS HOSPITAL,1216 SECOND STREET WEST,ROCHESTER,MN,55902,41
7,897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REH...,240036,ST CLOUD HOSPITAL,1406 6TH AVE NORTH,SAINT CLOUD,MN,56303,40
8,897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REH...,240057,ABBOTT NORTHWESTERN HOSPITAL,800 EAST 28TH STREET,MINNEAPOLIS,MN,55407,40
9,897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REH...,240053,PARK NICOLLET METHODIST HOSPITAL,6500 EXCELSIOR BLVD,SAINT LOUIS PAR,MN,55426,37


### 3 Get the geo location

In [7]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

Let's get geographical coordinates of Burnsville

In [8]:
address = 'Burnsville, MN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Burnsville are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Burnsville are 44.7677424, -93.2777226.


Let's get geographical coordinates of Fridley

In [9]:
address = 'Fridley, MN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Fridley are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Fridley are 45.0838291, -93.2590388.


Create location dataframe

In [10]:
# intialise data of lists 
data = {'Metro':['Minneapolis', 'Minneapolis'], 'City':['Burnsville','Fridley'], 'Latitude':[44.7677424, 45.0838291], 'Longitude':[-93.2777226, -93.2590388]} 
  
# create DataFrame 
metro_data = pd.DataFrame(data) 
  
# print the output 
metro_data

,Metro,City,Latitude,Longitude
0,Minneapolis,Burnsville,44.767742,-93.277723
1,Minneapolis,Fridley,45.083829,-93.259039


### 4 Explore cities with Foursquare API

Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OFPI1Z5114QOCBVXDX5UQXI4DHD3UKIT50D1GODYNGWT050L
CLIENT_SECRET:JFRXDKLKUCNMIX5NBDQ4RCYX1ASEFHZJUE3UIRKT2AIPZ3I2


Let's create a function to repeat the same process of exploring the above mentioned cities in the Minneapolis metro area

In [17]:
LIMIT = 500 # limit of number of venues returned by Foursquare API

radius = 5000 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function for each neighborhood/city and create a new dataframe called metro_venues.

In [18]:
metro_venues = getNearbyVenues(names=metro_data['City'],
                                   latitudes=metro_data['Latitude'],
                                   longitudes=metro_data['Longitude'])

Burnsville
Fridley


In [19]:
print(metro_venues.shape)
metro_venues.tail()

(198, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
193,Fridley,45.083829,-93.259039,Victoria's Secret,45.127242,-93.259901,Lingerie Store
194,Fridley,45.083829,-93.259039,SUBWAY,45.073862,-93.287272,Sandwich Place
195,Fridley,45.083829,-93.259039,LA Fitness,45.128206,-93.262205,Gym / Fitness Center
196,Fridley,45.083829,-93.259039,"Country Inn & Suites By Radisson, Brooklyn Cen...",45.074064,-93.313845,Hotel
197,Fridley,45.083829,-93.259039,Best Western Plus Minneapolis-Northwest,45.074156,-93.307095,Hotel


Let's check how many venues were returned for each neighborhood/city

In [20]:
metro_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Burnsville,98,98,98,98,98,98
Fridley,100,100,100,100,100,100


In [21]:
#Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(metro_venues['Venue Category'].unique())))

There are 97 uniques categories.


Analyze each neighborhood/city

In [22]:
# one hot encoding
metro_onehot = pd.get_dummies(metro_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
metro_onehot['Neighborhood'] = metro_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [metro_onehot.columns[-1]] + list(metro_onehot.columns[:-1])
metro_onehot = metro_onehot[fixed_columns]

metro_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Automotive Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Big Box Store,...,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Video Game Store,Video Store,Warehouse Store,Wine Bar,Wine Shop
0,Burnsville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,Burnsville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Burnsville,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,Burnsville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Burnsville,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [23]:
metro_grouped = metro_onehot.groupby('Neighborhood').mean().reset_index()

print('new shape',metro_grouped.shape)
metro_grouped.head()

new shape (2, 98)


,Neighborhood,American Restaurant,Asian Restaurant,Automotive Shop,Bakery,Bar,Baseball Field,Basketball Court,Beach,Big Box Store,...,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Trail,Video Game Store,Video Store,Warehouse Store,Wine Bar,Wine Shop
0,Burnsville,0.040816,0.020408,0.010204,0.010204,0.010204,0.00,0.010204,0.010204,0.010204,...,0.010204,0.010204,0.010204,0.00,0.010204,0.00,0.00,0.010204,0.010204,0.010204
1,Fridley,0.040000,0.010000,0.000000,0.030000,0.020000,0.01,0.000000,0.000000,0.010000,...,0.000000,0.020000,0.000000,0.01,0.010000,0.01,0.01,0.000000,0.000000,0.000000


Let's print each neighborhood along with the top 20 most common venues

In [24]:
num_top_venues = 20

for hood in metro_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = metro_grouped[metro_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Burnsville----
                        venue  freq
0                 Coffee Shop  0.06
1         American Restaurant  0.04
2              Sandwich Place  0.04
3               Grocery Store  0.04
4          Italian Restaurant  0.03
5          Mexican Restaurant  0.03
6                 Pizza Place  0.03
7                        Lake  0.02
8              Lingerie Store  0.02
9                   Disc Golf  0.02
10                       Park  0.02
11             Cosmetics Shop  0.02
12          Convenience Store  0.02
13           Asian Restaurant  0.02
14       Gym / Fitness Center  0.02
15                   Pharmacy  0.02
16                Gas Station  0.02
17               Burger Joint  0.02
18              Movie Theater  0.01
19  Middle Eastern Restaurant  0.01


----Fridley----
                   venue  freq
0            Pizza Place  0.06
1         Sandwich Place  0.05
2   Fast Food Restaurant  0.05
3           Liquor Store  0.05
4            Coffee Shop  0.05
5    American Restaur

Let's create a new dataframe and display the top 10 vanues for each neighborhood/city

In [25]:
# First, let's write a function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = metro_grouped['Neighborhood']

for ind in np.arange(metro_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(metro_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Burnsville,Coffee Shop,American Restaurant,Sandwich Place,Grocery Store,Pizza Place,Italian Restaurant,Mexican Restaurant,Gas Station,Lake,Disc Golf
1,Fridley,Pizza Place,Liquor Store,Fast Food Restaurant,Sandwich Place,Coffee Shop,Mexican Restaurant,American Restaurant,Bakery,Bookstore,Thai Restaurant


In [26]:
metro_merged = metro_data

#merge metro_grouped with metro_data to add latitude/longitude for each neighborhood
metro_merged = metro_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='City')

metro_merged.head() # check the last columns!

,Metro,City,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Minneapolis,Burnsville,44.767742,-93.277723,Coffee Shop,American Restaurant,Sandwich Place,Grocery Store,Pizza Place,Italian Restaurant,Mexican Restaurant,Gas Station,Lake,Disc Golf
1,Minneapolis,Fridley,45.083829,-93.259039,Pizza Place,Liquor Store,Fast Food Restaurant,Sandwich Place,Coffee Shop,Mexican Restaurant,American Restaurant,Bakery,Bookstore,Thai Restaurant


Finally, create a map of Minneapolis

In [27]:
# create map of Minneapolis(metro) using latitude and longitude values
address = 'Minneapolis, MN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Minneapolis are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Minneapolis are 44.9706705, -93.268217735811.


In [28]:
latitude=44.9706705
longitude=-93.268217735811

map_metro = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(metro_merged['Latitude'], metro_merged['Longitude'], metro_merged['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_metro)

    
map_metro